# Метрики и их царство:

In [1]:
import pandas as pd

Датасет будет прикреплён в виде ссылки так как он слишком большой для github:
[Ссылка](https://drive.google.com/file/d/1Ia3k1MoOOxrpJ8p7JOtNyQZGkE3ZumZj/view?usp=sharing).

### Введение и план:
____

Являясь продуктовым аналитиком, я постоянно сталкиваюсь с метриками и считаю, что важно не только просто знать, но уметь их правильно применять.
Именно для этого я и создаю данный проект, so,
Let's get started :)


<p align="center">
  <img width="600" height="327" src="metrics.jpg">
</p>

Для начала давайте разберёмся какие вообще бывают метрики и для чего он нужны для аналитиков:
- Метрики — это количественно измеряемые показатели, которые позволяют компаниям, командам или предприятиям определять и отслеживать успешность продукта или бизнес-деятельности. Используются для выявления проблем, отслеживания выполнения целей и принятия обоснованных решений.

При этом метрики принято делить на некоторые категории:
- Метрики действия (Actionable) — метрики, про которые известны чёткие законы связи метрик и прибыли, а еще те, на которые мы можем влиять непосредственно (то есть, имеем инструменты работы с этими метриками). Например, Retention rate, CAC.
- Метрики тщеславия (Vanity metrics) — это те метрики, которые напрямую не влияют на бизнес показатели проекта. Они красиво смотрятся, заставляют чувствовать себя хорошо, но не добавляют ничего к финансовой̆ отчетности вашего проекта. Например, количество лайков/репостов в соцсетях.

Как можно понять из определений,  превый тип метрик реально позволяет оценить состояние бизнеса, а второй лишь показывает различные значение, которые должны нас радовать (или нет), но какого-то insight мы в них не увидим.

А теперь я хочу приступить к практической части, в которой я выполню несколько задач по расчёту метрик. Как обычно начинаем с  описания данных и предосмотра данных:

#### Описание данных:
____

- date – дата совершения события

- event - событие:

    - app_install – установка приложения
    - app_start – открыть приложения
    - registration – зарегистрироваться
    - search – перейти на страницу поиска товаров (каталог)
    - open_item – открыть товар
    - choose_item – отправить товар в корзину
    - tap_basket – перейти в корзину
    - purchase – подтверждение покупки

- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь:

    - yandex-direct – Яндекс директ
    - google_ads – реклама в Google
    - vk_ads – реклама в ВК
    - instagram_ads – реклама в instagram
    - facebook_ads – реклама в facebook
    - referral – акция «приведи друга»

Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:
____

- В выгрузке только уникальные действия пользователей за каждый день

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку.

In [2]:
df = pd.read_csv('metrics.csv')
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
df.shape

(2747968, 8)

In [4]:
df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

В данном случае я не уберу Nan значения, так как здесь они означают, что некоторые пользователи просто ничего не купили.

Перед расчётами хочу оставить ссылки на вебинары и статью, которые помогут вам разобраться с основными видами метрик:
[Аналитика мобильных приложений, с чего начать?](https://www.youtube.com/watch?v=Ae0YO1izn84&list=PLo8_x3Axzvutlkh_ccSIvhJH7oBoLgOa_&index=21);
[Unit-экономика мобильный приложений](https://www.youtube.com/watch?v=mhEpREw4oKU&t=202s);
[Базовые продуктовые метрики](https://vc.ru/marketing/314555-bazovye-produktovye-metriki).
А теперь давайте приступим в задачам:

#### Рассчитайте MAU у февраля:
____

Формулы и определения:
- Дневная аудитория (DAU) — количество уникальных пользователей, которые зашли в приложение в течение суток;
- Недельная аудитория (WAU) - количество уникальных пользователей, которые зашли в приложение в течение недели;
- Месячная аудитория (MAU) — количество уникальных пользователей, которые зашли в приложение в течение месяца;
- avg. DAU/MAU = stickness ratio — коэффициент того, насколько часто пользователи, в течение одного периода, "остаются" в продукте.

In [5]:
df["date"] = pd.to_datetime(df["date"])

In [6]:
df["months"] = df["date"].dt.month

В данном случае, я сделал колонку, а не отбирал через query по дате, так как думаю, что колонка с месяцем нам ещё будет нужна.

In [7]:
df.query("months == 2").nunique()[4]

75032

В феврале в наш продукт пришли 75302 пользователя.

#### 2. Количество установок в январе:
____

In [8]:
df.query("months == 1 and event == 'app_install'").event.value_counts()

app_install    80297
Name: event, dtype: int64

#### Какой платный канал привлечения имеет самый высокий ROMI?
____

ROI отвечает в целом за окупаемость бизнеса, а ROMI и ROAS являются производными от ROI, но отвечают за расходы на маркетинг и рекламу соответственно.

Формулы:

- ROI = (доходы − расходы) / расходы

- ROMI = (Валовая прибыль − Маркетинговые расходы) / Маркетинговые расходы

- ROAS = (Валовая прибыль − расходы на рекламу) /расходы на рекламу

Дополнительно нам потребуются данные по затрате на рекламу:

- Яндекс – 10 491 707 руб.

- Гугл – 10 534 878 руб.

- Фейсбук – 8 590 498 руб.

- Инстаграм – 8 561626 руб.

- ВК – 9 553 531руб.

In [9]:
adv = df.groupby("utm_source",as_index=False).agg({"purchase_sum":"sum"})
adv

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


Внизу мне просто захотелось сделать эффектный вывод:

In [10]:
print(f"{(adv.loc[1][1] - 8590498) / 8590498} - Facebook",f"{(adv.loc[2][1] - 10534878)/ 10534878} - Google",f"{(adv.loc[3][1] - 8561626)/8561626} - Instagram",f"{(adv.loc[5][1] - 9553531)/9553531} - VK",f"{(adv.loc[6][1] - 10491707)/ 10491707} - Яндекс",sep="\n")

0.4259826380263403 - Facebook
0.22149264566708793 - Google
0.6990895187432854 - Instagram
0.7155596710786828 - VK
0.3263206835646478 - Яндекс


Как видно, ROMI  выше всего у VK.

#### С какого платного маркетингового канала пришло больше всего новых пользователей?
____

In [11]:
df.groupby("utm_source").agg({"device_id":pd.Series.nunique}).sort_values("device_id",ascending=False)

,device_id
utm_source,
-,52273
yandex-direct,40712
google_ads,38096
vk_ads,34086
instagram_ads,31048
facebook_ads,25959
referal,15926


Больше всего клиентов к нам пришло из Яндекса, но также можно обратить внимание, что и из Google кол-во практически тоже самое.

#### Пользователи, пришедшие с какого канала, имеют медианный первый чек выше?
____

In [12]:
first_purchase = df.query("event == 'purchase'").groupby("device_id").agg({"date":"min"}).reset_index().rename({"date":"first_purchase"},axis=1)

Я вычислил первую покупку у каждого юзера, который он сделал, при этом нам надо вычислить медианный первый чек и для этого нужно сджоинить её к основному фрейму и произвести расчёты:

In [13]:
purchase_df = first_purchase.merge(df,how="left",on="device_id")

In [14]:
purchase_df = purchase_df.query("event == 'purchase' and first_purchase == date")

In [15]:
purchase_df.groupby("utm_source").agg({"purchase_sum":"median"}).sort_values("purchase_sum",ascending=False)

,purchase_sum
utm_source,
-,398.5
referal,395.5
instagram_ads,393.5
vk_ads,393.0
yandex-direct,392.5
google_ads,390.5
facebook_ads,389.0


Первый медианный чек выше всего у пользователей, которые пришли к нам балгодаря реферальной программе.

#### Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?
____

Подсказка:

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение


В ходе доклада, мы уже затронули большое кол-во метрик, но важно не только их знать и понимать, а также определять является ли метрика хорошей:

Критерии хорошей метрики:
____

- Доступная для понимания — вся команда понимает, что значит эта метрика, так как она незамысловата;
- Чувствительная — метрика сразу отразит какие-то проблемы в одном или нескольких направлениях;
- Actionable — метрика несет смысл, и ее динамика или сравнение с другими метриками нам о чём-то говорит и меняет наше поведение;
- Быстрая — нам не нужно ждать конца месяца, чтобы посчитать эту метрику. Ценятся те метрики, которые можно измерять риалтайм и реагировать;
- Релевантная — относится к нашим целям и тому, что мы хотим понять или узнать.

Вроде бы всё понятно, но что означает чувствительность метрики ?
Чувствительность метрики - это ситуация, когда метрики даже при малейших изменений начнётся "прокрашиваться" на синтетичных A/A тестах, но я настоятельно рекомендую посмотреть вебинары, прикреплённые ниже:
- [Виталий Черемисинов, EXPF, - Как искать прокси-метрики в продуктах](https://www.youtube.com/watch?v=fSRKOr3L6AI&t=498s);
- [Online meetup EXPF x Delivery Club (доклад Тимура из Авито)](https://www.youtube.com/watch?v=3eoIq8Csa1Y).

Ну а я смело начну выполнение задания:

In [16]:
df.utm_source.unique()

array(['-', 'vk_ads', 'referal', 'facebook_ads', 'google_ads',
       'instagram_ads', 'yandex-direct'], dtype=object)

In [17]:
yandex_a = df.query("event == 'purchase' and utm_source == 'referal'").groupby("device_id").agg({"date":"min"}).reset_index().rename({"date":"first_purchase"},axis=1)

In [18]:
yandex_b = df.query("event == 'app_start' and utm_source == 'referal'").groupby("device_id").agg({"date":"min"}).reset_index().rename({"date":"first_purchase"},axis=1)

In [19]:
yandex_a.shape[0] / yandex_b.shape[0]

0.3995227329816629

В данном случае я думаю логичнее будет просто подставлять разные каналы в одну и туже переменную и посмотреть на значение в print, так как если создавать для каждого канала свою переменную код будет очень большим:

In [20]:
print("Яндекс - 0.2954774363131648, Google - 0.29767405229444505, vk -  0.3628040728894627, Facebook - 0.3473822090380244, Instagram - 0.34665807698502177, Referral - 0.3995227329816629")

Яндекс - 0.2954774363131648, Google - 0.29767405229444505, vk -  0.3628040728894627, Facebook - 0.3473822090380244, Instagram - 0.34665807698502177, Referral - 0.3995227329816629


Как мы видим, самая низкая конверсия у Яндекса.

#### Как выбрать метрику для A/B теста ?
____

Для начала я хотел бы дать общие рекомендации при выборе метрики:
Проверочные вопросы для выбора продуктовых метрик
- Дают ли эти показатели реальную картину успеха нашего продукта?
- Влияют ли они на то, как мы относимся к продукту?
- Мотивируют ли они команду, которая создаёт продукт?

А также надо учитывать какая цель эксперимента:

1. Business (инвесторам):
    Метрики, которые важны инвесторам. Могут быть разные в зависимости от типа продукта и стадии его развития.
    Примеры: доля рынка (в объёме продаж), количество новых покупателей и так далее.

2. Margin (экономика):
    Все метрики, касающиеся юнит-экономики и маржинальности. Демонстрируют то, насколько рентабелен и перспективен продукт.
    Примеры: CAC, ARPU, LTV и так далее.

3. Loyality (лояльность):
    Метрики, отражающие лояльность пользователей к продукту.
    Примеры: retention, К-фактор, churn rate, NPS, средняя оценка приложения в app store/google play и так далее.

4. Value (ценность):
    Метрики, отражающие, что пользователь нашёл для себя ценность продукта.
    Примеры: прослушивание музыки больше N минут в день, N поездок за Х дней и так далее.

5. Quality (качество):
    Предполагаемое качество продукта / контента и тому подобное.
    Примеры: % брака, количество жалоб, отказы и так далее.

6. Marketing (маркетинг):
    Отражают всё, что связано с трафиком и каналами привлечения пользователей в продукт.
    Примеры: DAU, MAU, clicks, cost, installs.

Также я хотел бы порекомендовать несколько книг по продуктовой аналитки и A/B тестам:
- [Бизнес с нуля: Метод Lean Startup для быстрого тестирования идей и выбора бизнес-модели | Рис Эрик](https://www.ozon.ru/product/biznes-s-nulya-metod-lean-startup-dlya-bystrogo-testirovaniya-idey-i-vybora-biznes-modeli-ris-erik-254359868/?sh=RoJKJmdagg);
- [Доверительное А/В-тестирование](https://www.ozon.ru/product/doveritelnoe-a-v-testirovanie-224221240/?sh=RoJKJrnLbQ).

Ну а теперь возвращаемся в задачам:)

#### Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?
____

Подсказка:

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован.



In [21]:
registration = df.query("event == 'register'").groupby("device_id").agg({"date":"min"}).\
    reset_index().rename({"date":"date_of_registration"},axis=1)

In [22]:
df = df.merge(registration,how="left",on="device_id")

Воронка и конверсия:
____
- Воронка — последовательность действий пользователей до совершения ключевого действия. Показывает, как пользователи проходят через определенную последовательность действий в продукте, на каких шагах и какая часть из них отваливается.

- Конверсия (conversion rate, CR) — отношение числа пользователей, которые выполнили какое-либо целевое действие к общему числу пользователей.

- Абсолютная конверсия — это конверсия в n-ый шаг из первого (из 100% посетителей – 2,4% добавили в корзину)
Относительная конверсия — конверсия между двумя определенными шагами (из 1290 пользователей 60% (780) перешли корзину)
Важно разделять эти конверсии, они обе нам нужны и важны.

- Конверсионное окно (Conversion window) — это цикл целевого действия, то есть количество дней, за которое пользователь переходит из одного шага воронки в другой.


<p align="center">
  <img  src="conversion.png">
</p>

In [23]:
round((df.query("event == 'choose_item' and date > date_of_registration").shape[0] / df.query("event == 'search' and date > date_of_registration").shape[0]) * 100,2)

75.39

In [24]:
round((df.query("event == 'tap_basket' and date > date_of_registration").shape[0] / df.query("event == 'choose_item' and date > date_of_registration").shape[0]) * 100,2)

69.09

In [25]:
round((df.query("event == 'purchase' and date > date_of_registration").shape[0] / df.query("event == 'tap_basket' and date > date_of_registration").shape[0]) * 100,2)

33.89

#### Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей?
____

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [26]:
app_install = df.query("event == 'app_install'").groupby("device_id").agg({"date":"min"}).\
    reset_index().rename({"date":"date_of_installation"},axis=1)

In [27]:
purchase = df.query("event == 'purchase'").groupby("device_id").agg({"date":"min"}).\
    reset_index().rename({"date":"purchase"},axis=1)

In [28]:
app_install = app_install.merge(purchase,on="device_id",how="left")

In [29]:
app_install["difference"] = (app_install["purchase"] - app_install["date_of_installation"]).dt.days

In [30]:
cohort = app_install.query("difference <= 7")

Формула конверсии:
- Конверсия = (Количество посетителей с целевым действием/ Общее число посетителей) * 100%

In [31]:
cohort = cohort.groupby("date_of_installation",as_index=False).agg({"device_id":"count"}).rename({"device_id":"less_than_7_days"},axis=1)

In [32]:
df2 =app_install\
        .groupby('date_of_installation')\
        .agg({'device_id': 'count'})\
        .rename(columns={'device_id': 'total_amount'})\
        .reset_index()

In [33]:
cohort = cohort.merge(df2, on='date_of_installation')

In [34]:
cohort['CR'] = round((cohort['less_than_7_days'] / cohort['total_amount']) * 100,1)

In [35]:
cohort.sort_values("CR",ascending=False)

,date_of_installation,less_than_7_days,total_amount,CR
0,2020-01-01,1408,3579,39.3
8,2020-01-09,558,1424,39.2
14,2020-01-15,1650,4310,38.3
13,2020-01-14,1973,5173,38.1
1,2020-01-02,1186,3144,37.7
...,...,...,...,...
77,2020-03-18,235,1171,20.1
88,2020-03-29,223,1117,20.0
81,2020-03-22,251,1261,19.9
87,2020-03-28,209,1091,19.2


### Дополнительно:
____
Также помимо посчитанных метрик, я также хочу упомянуть метрики, которые показывает показатели доходов:

1. ARPPU (Average revenue per paying user). Средний доход на платящего пользователя (то есть клиента). Состоит из того, насколько часто пользователь покупает, и насколько большой средний чек покупки.

   - ARPPU=Число повторных покупок⋅Средний чек

2. Для того, чтобы посчитать повторные покупки, нужно разделить общее количество покупок на общее количество покупателей,

   - Повторные покупки = Число покупок / Число клиентов

3. Средний чек — это доход, разделённый на количество покупок.

   - Средний чек = Доход / Число покупок

    Перемножаем две получившиеся позиции и получаем итог — ARPPU :)


4. ARPU (Average revenue per user). Cредний доход с привлечённого пользователя. Для удобства и принятия решений разделим формулу ARPU на составные части.

   - ARPU=Конверсия⋅ARPPU

   - Конверсия = Число клиентов / Число пользователей

5. Также ARPPU можно рассчитать вот так:

   - ARPPU = Доход / Число клиентов

6. LTV (lifetime value) Пожизненная ценность клиента. Отражает, сколько в среднем приносит пользователь денег в продукт до того, как покинет его. Смысл LTV состоит в том, чтобы прикинуть, сколько можно потратить на привлечение нового клиента на ранней стадии, в отношении вероятной прибыли от одного человека. Рассчитывается как произведение среднего дохода на пользователя (ARPU) и средней продолжительности жизни клиента (Lifetime).

   - LTV=ARPU⋅Lifetime

Также я оставлю дополнительные материалы по LTV:
[Василий Сабиров (devtodev) - Важен каждый: почему нужно следить за полной ценностью клиентов](https://www.youtube.com/watch?v=rwvc3KHcuE8);
[5 Simple Ways to Calculate Customer Lifetime Value](https://medium.com/swlh/5-simple-ways-to-calculate-customer-lifetime-value-5f49b1a12723).

Помимо всего того, что я вам дал, я хочу добавить ещё несколько метрик:

Показатели расходов:
____

1. CPAcq (Cost per acquisition) – стоимость привлечения пользователей. Метрика показывает, во сколько компании обходится один пользователь, заходящий в продукт.

   - CPAcq = Расходы на привлечение / Число пользователей

2. CAC – стоимость привлечения клиента. Метрика показывает, во сколько компании обходится один клиент (то есть именно клиент — пользователь, совершивший целевое действие).  CPAcq и САС – это разные определения и метрики, не стоит их путать.

   - CAC = Расходы на привлечение / Число клиентов



Сравнивая LTV и CAC, мы можем определить, рентабелен маркетинг продукта или нет (то есть окупаются ли расходы на привлечение пользователей).

Для принятия решений связанных с масштабированием, в расчёте CPAcq важно учитывать не всех пользователей, а только тех, которых мы привлекли, потратив деньги на привлечение. То есть только неорганический трафик.

### LTV >> CAC

Прочие метрики:
____

Можно выделить очень много разных метрик (главное в них не запутаться). Давайте посмотрим на некоторые специфичные метрики.

1. NPS (net promoter score) — метрика уровня лояльности пользователей. Измеряется с помощью опросников, отражается в процентах. NPS измеряет количество постоянных клиентов, которые могут порекомендовать продукт (промоутеры), и тех клиентов, которые его ненавидят.

   - NPS=% of promoters − % of detractors

2. Оценка удовлетворенности клиентов (customer satisfaction score, CSAT) — измеряет общий уровень недовольства пользователя по поводу определенного продукта или функции. Пользователей просят оценить продукт или услугу по шкале 1-3, 1-5 или 1- 10. CSAT рассчитывается путем суммирования баллов и деления его на количество респондентов.

CSAT = сумма положительных оценок / количество всех оценок ⋅ 100%

3. Фактор виральности. Чем больше K-фактор, тем стремительнее распространяется ваш продукт по сети и тем он более вирален, то есть нравится аудитории.

  - К−фактор=X⋅Y⋅Z

    1. X — процент пользователей, которые пригласили новых пользователей

    2. Y — среднее число людей, которых пригласил единичный пользователь

    3. Z — число пользователей, принявших приглашение

4. Средняя продолжительность сессии (average session length, ASL) – среднее арифметическое длин всех сессий.

   - ASL= T / N

    1. T — суммарная продолжительность сессий за период

    2. N — общее количество сессий за тот же период

5. Отказы — количество визитов, в которые посетитель просмотрел не более одной страницы и пробыл на сайте менее 15 секунд.

6. Глубина просмотра – среднее количество страниц, просмотренных посетителями сайта за один визит.

7. Количество сеансов (сессий) на пользователя — помогает понять как часто пользователи возвращаются и используют сайт.

### Выводы:
____

- Данный доклад я сделал не только для вас, но и для себя в качестве опоры на формулы и тем самым данный проект является справочником.